<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Однородное магнитное поле

### Examples - Electromagnetism
<section class="post-meta">
By Eilif Sommer Øyre, Niels Henrik Aase, Thorvald Ballestad, and Jon Andreas Støvneng
</section>
Last edited: September 1st 2019

___

## Вступление

Заряженная частица, движущаяся в магнитном поле, будет испытывать магнитную силу, управляемую законом силы Лоренца. Эта сила всегда перпендикулярна направлению движения частицы. Если частица движется в однородном магнитном поле и скорость частицы нормальна к направлению поля, частица в конечном итоге движется по кругу. Теперь, если направление поля не нормально скорости, частица начинает двигаться по спирали, если только ее движение не полностью параллельно полю, в котором нет силы Лоренца, и частица движется по совершенно прямой линии. На этом уроке мы попытаемся смоделировать два предыдущих сценария путем численного интегрирования уравнений движения частицы с использованием метода Эйлера.

## Теория
### Закон силы Лоренца
Частица с зарядом $Q$ и массой $m_p$, движущаяся со скоростью $\mathbf{v}$ в магнитном поле $\mathbf{B}$, испытывает силу 

\begin{equation}
    \mathbf{F}_{mag} = Q(\mathbf{v} \times \mathbf{B})
\label{Lorentz} \quad (1)
\end{equation}

известную как *сила Лоренца* [[1]](#rsc). Здесь жирные символы представляют векторы, в то время как обычные символы представляют скаляры.

### Дискретизация
Сила Лоренца может быть записана в виде обыкновенного дифференциального уравнения второго порядка (ОДУ)

\begin{equation}
m_p\frac{d^2\mathbf{x}}{dt^2} = Q\big(\frac{d\mathbf{x}}{dt} \times \mathbf{B}\big),
\label{LorentzODE} \quad (2)
\end{equation}

где компоненты скорости записываются как производная по времени от компонентов $v_i = dx_i/dt$. Левая часть уравнения, конечно, является вторым законом Ньютона, где $d^2\mathbf{x}/dt^2$ - это ускорение частицы, записанное как вторая производная положения по времени.  Разложив векторы, (2) может быть сведено к трем наборам из двух ОДУ первого порядка.

\begin{equation}
\begin{aligned}
    &\frac{dx}{dt} = v_x\\[4pt]
    &\frac{dv_x}{dt} = \frac{Q}{m_{p}}(v_yB_z - v_zB_y)\\[10pt]
    &\frac{dy}{dt} = v_y\\[4pt]
    &\frac{dv_y}{dt}= \frac{Q}{m_{p}}(v_zB_x - v_xB_z)\\[10pt]
    &\frac{dz}{dt} = v_z\\[4pt]
    &\frac{dv_z}{dt} = \frac{Q}{m_{p}}(v_xB_y - v_yB_x) .
\end{aligned}
\label{LorentzComponents} \quad (3)
\end{equation}

Используя численный метод, такой как метод Эйлера или четвертый порядок Рунге-Кутты, мы можем найти приближенные решения для ОДУ, используя заданный временной шаг, $\Delta t$, и найти движение заряженной частицы в магнитном поле.

## Численная достоверность

### Сохранение энергии

Поскольку сила Лоренца является перекрестным произведением скорости частицы и магнитного поля, ее направление всегда перпендикулярно скорости частицы. Следовательно, сила Лоренца не действует на частицу, и кинетическая энергия частицы сохраняется. Затем проводится проверка численной достоверности для проверки отклонения начальной и конечной кинетической энергии.

### Larmor radius

Как указано во введении, заряженная частица будет двигаться по кругу, если она имеет составляющую скорости, перпендикулярную внешнему магнитному полю. Это называется вращением Лармора, а гирорадиус (или радиус Лармора), радиус окружности, задается [[2]](#rsc)

\begin{equation}
r_L = \frac{mv_{\perp}}{|Q| B} ,
\label{larmorRadius} \quad (4)
\end{equation}

где $v_{\perp}$ - скорость частицы, перпендикулярной магнитному полю, с величиной $B$.

## Реализация

In [ ]:
# First, import all necessary library packages
import numpy as np              # For math and array manipulation
import matplotlib.pyplot as plt # For plotting
from mpl_toolkits.mplot3d import Axes3D # For 3D-plotting
plt.style.use("bmh")            # Set a particular style on the plots
# "Magic" command to adapt matplotlib plots to jupyter notebook
%matplotlib inline              
figsize = (6, 6)                # Tuple tho hold figure size of plots
dpi = 100                       # Dots per inch on plots (resolution)

Заряженную частицу, движущуюся в однородном магнитном поле, можно легко смоделировать с помощью метода Эйлера для уравнений (3). Ниже мы определяем функцию для правой части уравнений в (3), а также функцию для одного явного шага Эйлера с учетом текущего положения и скорости.

In [ ]:
def RHS(M, RHSargs):
    """ Returns the right hand side of the sets of ordinary 
        differential equations.
    Arguments:
        M         1x6 array containing the position and
                  velocity of the particle in cartesian
                  coordinates.
        RHSargs   1x5 array containing the arguments/variables
                  of the right hand side of the Lorentz force.
    """
    x, y, z, vx, vy, vz = M
    Q, mass, Bx, By, Bz = RHSargs
    ddt_vx = Q/mass*(vy*Bz - vz*By)
    ddt_vy = Q/mass*(vz*Bx - vx*Bz)
    ddt_vz = Q/mass*(vx*By - vy*Bx)
    ddt_x = vx
    ddt_y = vy
    ddt_z = vz
    return ddt_x, ddt_y, ddt_z, ddt_vx, ddt_vy, ddt_vz


def eulerStep(M, dt, RHS, RHSargs):
    """ Performs one step of the Euler method."""
    x, y, z, vx, vy, vz = M # Extract positional and velocity components from matrix
    dx, dy, dz, dvx, dvy, dvz = RHS(M, RHSargs) # Calculate right hand side of ODE's
    # Increment each component using Euler's method
    x = x + dx*dt
    y = y + dy*dt
    z = z + dz*dt
    vx = vx + dvx*dt
    vy = vy + dvy*dt
    vz = vz + dvz*dt
    return x, y, z, vx, vy, vz

На данный момент мы хотим, чтобы частица двигалась в однородном магнитном поле с начальной скоростью, перпендикулярной направлению поля.

In [ ]:
Q = 1                   # Particle charge
mass = 1                # Particle mass
Bx = 10                 # Magnetic field x-component magnitude
By = 0                  # Magnetic field y-component magnitude
Bz = 0                  # Magnetic field z-component magnitude
h = 1e-4                # Time step for numerical integration
tMax = 2                # Total duration of integration
n = int(tMax/h)         # Number of datapoints


# Array of arguments (parameters) to the RHS function
RHSargs = np.array([Q, mass, Bx, By, Bz])

# Initialising data matrix
M = np.zeros((6, n))

# Initial position and velocity components
x0 = 0    # x-component magnitude of positional vector
y0 = 0    # y-component magnitude of positional vector
z0 = 0    # z-component magnitude of positional vector
vx0 = 0   # x-component magnitude of velocity vector
vy0 = 10  # y-component magnitude of velocity vector
vz0 = 0   # z-component magnitude of velocity vector
M[:, 0] = np.array([x0, y0, z0, vx0, vy0, vz0]) # Storing the components in the data matrix

# The matrix M will have the following form when completly filled with data:
# 
# M[ROW, COLUMN]
#
#           n COLUMNS 
#    -----------------------------------
# 6 | x0   x1   x2   ...  x_n-2   x_n-1
#   | y0   y1   y2   ...  y_n-1   y_n-1
# R | z0   z1   z2   ...  z_n-1   z_n-1
# O | vx0  vx1  vx2  ...  vx_n-1  vx_n-1
# W | vy0  vy1  vy2  ...  vy_n-1  vy_n-1
# S | vz0  vz1  vz2  ...  vz_n-1  vz_n-1
# 
# Writing ":" in M[ROWS, COLUMNS] such as M[:, 0] returns an array containing
# the first column. M[0, :] returns the first row
#

# Integrate particle trajectory
for i in range(n-1):
    M[:, i+1] = eulerStep(M[:, i], h, RHS, RHSargs)
    
# Calculating deviation in kinetic energy
# Check out scipy.org for a description of the numpy-function numpy.linalg.norm(),
# here we use it to calculate the absolute value of the velocity by squaring its components.
Ke_initial = 0.5*mass*np.absolute(np.linalg.norm(M[3:6, 0], axis=0))**2 # mv^2/2 (formula for kinetic energy)
Ke_final = 0.5*mass*np.absolute(np.linalg.norm(M[3:6, -1], axis=0))**2
energyDeviation = np.abs(Ke_final - Ke_initial)/Ke_initial*100
print("Initial to final energy deviation:\t\t%.3f percent"%(energyDeviation))

# Plot path
plt.figure(figsize=(6, 6), dpi=dpi)
plt.plot(M[1], M[2])
plt.xlabel('y', size=10)
plt.ylabel('z', size=10)
plt.title("Charged particle in uniform B-field", size=10)

plt.show()

Частица движется по кругу в плоскости $yz$, как и ожидалось в уравнении (1). Наши параметры дают радиус Лармора $r_g = 1$ из уравнения (4). При беглом взгляде на ось выше это кажется точным. 

Что касается сохранения энергии, мы видим, что кинетическая энергия отклоняется на 2\% $от своего начального значения, что указывает на то, что временной шаг был примерно подходящим для этого интервала интегрирования. Однако при интегрировании в течение более длительного периода (больший "tmax") это отклонение может вырасти до больших размеров из-за накапливающейся численной ошибки на каждом шаге Эйлера, и численная достоверность ослабевает. Возможно, было бы разумно провести тест на сходимость с использованием разных временных шагов и сравнить общую ошибку.

In [ ]:
# Creating an array of timeSteps with logarithmic spacing
timeSteps = np.logspace(-7, -2.5, 30)
# Array to hold deviations for each time step
deviations = np.zeros((len(timeSteps)))

Ke_initial = 0.5*mass*np.absolute(np.linalg.norm(M[3:6, 0], axis=0))**2

for k in range(len(timeSteps)):
    h = timeSteps[k]
    n = int(tMax/h)   # Number of datapoints
    # Initialising data matrix
    M = np.zeros((6, n))
    M[:, 0] = np.array([x0, y0, z0, vx0, vy0, vz0]) # Storing the components in the data matrix

    # Integrate particle trajectory
    for i in range(n-1):
        M[:, i+1] = eulerStep(M[:, i], h, RHS, RHSargs)

    # Calculating deviation in kinetic energy
    Ke_final = 0.5*mass*np.absolute(np.linalg.norm(M[3:6, -1], axis=0))**2
    energyDeviation = np.abs(Ke_final - Ke_initial)/Ke_initial*100
    deviations[k] = energyDeviation

plt.figure(figsize=(8, 4), dpi=dpi)
plt.loglog(timeSteps, deviations)
plt.xlabel('time step, [s]')
plt.ylabel('Initial to final energy deviation, percent')
plt.title('Numerical convergence test')
plt.show()

Из приведенного выше графика мы видим, что ошибка в кинетической скорости имеет первый порядок на временном шаге $\Delta t$. Этот результат также указывает на то, что временного шага ниже $10^{-5}\mathrm{s}$ достаточно, чтобы получить отклонение ниже $0,1\%$.

Теперь, как выглядит траектория частицы, если начальная скорость находится как в направлении $x$, так и в направлении $y$?

In [ ]:
vx0 = 10             # Resetting vx0 to 10
h = 1e-5
n = int(tMax/h)      # Recalculating number of datapoints
M = np.zeros((6, n)) # Re-initialising data matrix
M[:, 0] = np.array([x0, y0, z0, vx0, vy0, vz0])

for i in range(n-1):
    M[:, i+1] = eulerStep(M[:, i], h, RHS, RHSargs)
# Calculating deviation in kinetic energy
Ke_initial = 0.5*mass*np.absolute(np.linalg.norm(M[3:6, 0], axis=0))**2 # mv^2/2 (formula for kinetic energy)
Ke_final = 0.5*mass*np.absolute(np.linalg.norm(M[3:6, -1], axis=0))**2
energyDeviation = np.abs(Ke_final - Ke_initial)/Ke_initial*100
print("Initial to final energy deviation:\t\t%.3f percent"%(energyDeviation))
    
fig = plt.figure(figsize=(6,6), dpi=100)
ax = fig.add_subplot(111, projection='3d')
# Plotting guiding line
ax.plot(M[0], M[0]*0, np.ones(len(M[0]))*-1, '--', linewidth=1)
# Plotting particle path
ax.plot(M[0], M[1], M[2])
ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$y$")
ax.set_zlabel(r"$z$")
ax.set_title("Charged particle in uniform B-field")

plt.show()

Мы получаем спиральное движение! Скорость в направлении $x$ остается неизменной как следствие поперечного произведения в законе силы Лоренца. Синяя линия - это движение центра вращения Лармора. Это называется дрейфом плоскости движения.

___
<a id="rsc"></a>
## Resources and Further Reading
<a>[1]</a>: Chen, F. 1984, Introduction to plasma physics and controlled fusion. Volume 1: Plasma physics (New York: Plenum Press) <br />
<a>[2]</a>: [Bittencourt, J. 2004, Fundamentals of plasma physics (New York: Springer-Verlag)](http://home.zcu.cz/~kozakt/MPPL/literatura/Bittencourt%20-%20Fundamentals%20of%20Plasma%20Physics.pdf) <br />
<a>[3]</a>:  https://habr.com/ru/post/429790/